In [76]:
import os
import torch
import pandas as pd
import numpy as np
import PIL
import torch.nn as nn
import torch.optim as optim
from torchvision import models
from PIL import Image
from sklearn import svm
from sklearn.metrics import accuracy_score
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from efficientnet_pytorch import EfficientNet
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import TensorDataset
from sklearn.metrics import confusion_matrix, classification_report

In [20]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [13]:
three_dataset = pd.read_csv("three_dataset.csv")

In [14]:
three_dataset["skin_tone_category"].value_counts()

skin_tone_category
dark      233
light     233
medium    233
Name: count, dtype: int64

In [15]:
three_dataset_light = three_dataset[233:466]
three_dataset_medium = three_dataset[466:699]
three_dataset_dark = three_dataset[0:233]

In [16]:
model = EfficientNet.from_pretrained('efficientnet-b0')
model.eval()

Loaded pretrained weights for efficientnet-b0


EfficientNet(
  (_conv_stem): Conv2dStaticSamePadding(
    3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False
    (static_padding): ZeroPad2d((0, 1, 0, 1))
  )
  (_bn0): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
  (_blocks): ModuleList(
    (0): MBConvBlock(
      (_depthwise_conv): Conv2dStaticSamePadding(
        32, 32, kernel_size=(3, 3), stride=[1, 1], groups=32, bias=False
        (static_padding): ZeroPad2d((1, 1, 1, 1))
      )
      (_bn1): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
      (_se_reduce): Conv2dStaticSamePadding(
        32, 8, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_se_expand): Conv2dStaticSamePadding(
        8, 32, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_project_conv): Conv2dStaticSamePadding(
        32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False
    

In [17]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),       # Resize to 224x224
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],      # ImageNet means
        std=[0.229, 0.224, 0.225]        # ImageNet stds
    )
])

In [18]:
img_dir = 'ddi + pad'
img_files = [f for f in os.listdir(img_dir) if f.endswith(('.png', '.jpg', '.jpeg'))]

In [25]:
def image_resize(image):
    original_size = image.size
    target_size = (224, 224)
    ratio = min(target_size[0] / original_size[0], target_size[1] / original_size[1])
    new_size = (int(original_size[0] * ratio), int(original_size[1] * ratio))
    resized_image = image.resize(new_size, Image.Resampling.LANCZOS)

    new_image = Image.new("RGB", target_size, (255, 255, 255))
    paste_position = ((target_size[0] - new_size[0]) // 2, (target_size[1] - new_size[1]) // 2)
    new_image.paste(resized_image, paste_position)

    return new_image

In [28]:
folder = "ddi + pad"
file_names = three_dataset_light["img_id"]
images = []
for file in file_names:
    path = os.path.join(folder, file)
    image = PIL.Image.open(path)
    #print(image.size) # Returns (width, height)
    images.append(np.array(image_resize(image)).flatten())
X_light = np.stack(images)
print(X_light.shape)

(233, 150528)


In [ ]:
y_light = three_dataset_light["label"]
le = LabelEncoder()
y_light_encoded = le.fit_transform(y_light)

In [74]:
print(le.classes_)

['melanoma' 'not-melanoma']


In [ ]:
X_light_train, X_light_test, y_light_train, y_light_test = train_test_split(X_light, y_light_encoded, test_size=0.2, stratify=y_light_encoded, random_state=42)

In [99]:
light_train = pd.concat([three_dataset_light[three_dataset_light["label"] == "melanoma"][:18],three_dataset_light[three_dataset_light["label"] != "melanoma"][:168]], ignore_index=True)
light_test = pd.concat([three_dataset_light[three_dataset_light["label"] == "melanoma"][18:],three_dataset_light[three_dataset_light["label"] != "melanoma"][168:]], ignore_index=True)

In [100]:
folder = "ddi + pad"
file_names = light_train["img_id"]
images = []
for file in file_names:
    path = os.path.join(folder, file)
    image = PIL.Image.open(path)
    #print(image.size) # Returns (width, height)
    images.append(np.array(image_resize(image)).flatten())
X_light_train = np.stack(images)

In [101]:
folder = "ddi + pad"
file_names = light_test["img_id"]
images = []
for file in file_names:
    path = os.path.join(folder, file)
    image = PIL.Image.open(path)
    #print(image.size) # Returns (width, height)
    images.append(np.array(image_resize(image)).flatten())
X_light_test = np.stack(images)

In [102]:
y_light_train = light_train["label"]
le = LabelEncoder()
y_light_train_encoded = le.fit_transform(y_light_train)

In [103]:
y_light_test = light_test["label"]
le = LabelEncoder()
y_light_test_encoded = le.fit_transform(y_light_test)

In [93]:
X_light_train["label"].value_counts()

label
not-melanoma    168
melanoma         18
Name: count, dtype: int64

In [94]:
X_light_test["label"].value_counts()

label
not-melanoma    32
melanoma        15
Name: count, dtype: int64

In [116]:
medium_train = pd.concat([three_dataset_medium[three_dataset_medium["label"] == "melanoma"][:23],three_dataset_medium[three_dataset_medium["label"] != "melanoma"][:168]], ignore_index=True)
medium_test = pd.concat([three_dataset_medium[three_dataset_medium["label"] == "melanoma"][23:],three_dataset_medium[three_dataset_medium["label"] != "melanoma"][168:]], ignore_index=True)

In [110]:
folder = "ddi + pad"
file_names = medium_train["img_id"]
images = []
for file in file_names:
    path = os.path.join(folder, file)
    image = PIL.Image.open(path)
    #print(image.size) # Returns (width, height)
    images.append(np.array(image_resize(image)).flatten())
X_medium_train = np.stack(images)

folder = "ddi + pad"
file_names = medium_test["img_id"]
images = []
for file in file_names:
    path = os.path.join(folder, file)
    image = PIL.Image.open(path)
    #print(image.size) # Returns (width, height)
    images.append(np.array(image_resize(image)).flatten())
X_medium_test = np.stack(images)

y_medium_train = medium_train["label"]
le = LabelEncoder()
y_medium_train_encoded = le.fit_transform(y_medium_train)

y_medium_test = medium_test["label"]
le = LabelEncoder()
y_medium_test_encoded = le.fit_transform(y_medium_test)

X_medium_train_tensor = torch.tensor(X_medium_train, dtype=torch.float32)
X_medium_test_tensor = torch.tensor(X_medium_test, dtype=torch.float32)
y_medium_train_tensor = torch.tensor(y_medium_train_encoded, dtype=torch.long)
y_medium_test_tensor = torch.tensor(y_medium_test_encoded, dtype=torch.long)

X_medium_train_tensor = torch.stack([transform(Image.fromarray(img.astype('uint8')).convert("RGB")) for img in X_medium_train])
X_medium_test_tensor = torch.stack([transform(Image.fromarray(img.astype('uint8')).convert("RGB")) for img in X_medium_test])

train_medium_dataset = TensorDataset(X_medium_train_tensor, y_medium_train_tensor)
test_medium_dataset = TensorDataset(X_medium_test_tensor, y_medium_test_tensor)

train_medium_loader = DataLoader(train_medium_dataset, batch_size=32, shuffle=True)
test_medium_loader = DataLoader(test_medium_dataset, batch_size=32)

In [ ]:
dark_train = pd.concat([three_dataset_dark[three_dataset_dark["label"] == "melanoma"][:18],three_dataset_dark[three_dataset_dark["label"] != "melanoma"][:168]], ignore_index=True)
dark_test = pd.concat([three_dataset_dark[three_dataset_dark["label"] == "melanoma"][18:],three_dataset_dark[three_dataset_dark["label"] != "melanoma"][168:]], ignore_index=True)

In [ ]:
folder = "ddi + pad"
file_names = dark_train["img_id"]
images = []
for file in file_names:
    path = os.path.join(folder, file)
    image = PIL.Image.open(path)
    #print(image.size) # Returns (width, height)
    images.append(np.array(image_resize(image)).flatten())
X_dark_train = np.stack(images)

folder = "ddi + pad"
file_names = dark_test["img_id"]
images = []
for file in file_names:
    path = os.path.join(folder, file)
    image = PIL.Image.open(path)
    #print(image.size) # Returns (width, height)
    images.append(np.array(image_resize(image)).flatten())
X_dark_test = np.stack(images)

y_dark_train = dark_train["label"]
le = LabelEncoder()
y_dark_train_encoded = le.fit_transform(y_dark_train)

y_dark_test = dark_test["label"]
le = LabelEncoder()
y_dark_test_encoded = le.fit_transform(y_dark_test)

X_dark_train_tensor = torch.tensor(X_dark_train, dtype=torch.float32)
X_dark_test_tensor = torch.tensor(X_dark_test, dtype=torch.float32)
y_dark_train_tensor = torch.tensor(y_dark_train_encoded, dtype=torch.long)
y_dark_test_tensor = torch.tensor(y_dark_test_encoded, dtype=torch.long)

X_dark_train_tensor = torch.stack([transform(Image.fromarray(img.astype('uint8')).convert("RGB")) for img in X_dark_train])
X_dark_test_tensor = torch.stack([transform(Image.fromarray(img.astype('uint8')).convert("RGB")) for img in X_dark_test])

train_dark_dataset = TensorDataset(X_dark_train_tensor, y_dark_train_tensor)
test_dark_dataset = TensorDataset(X_dark_test_tensor, y_dark_test_tensor)

train_dark_loader = DataLoader(train_dark_dataset, batch_size=32, shuffle=True)
test_dark_loader = DataLoader(test_dark_dataset, batch_size=32)

In [117]:
X_light_train_tensor = torch.tensor(X_light_train, dtype=torch.float32)
X_light_test_tensor = torch.tensor(X_light_test, dtype=torch.float32)
y_light_train_tensor = torch.tensor(y_light_train_encoded, dtype=torch.long)
y_light_test_tensor = torch.tensor(y_light_test_encoded, dtype=torch.long)

In [118]:
X_light_train_tensor = torch.stack([transform(Image.fromarray(img.astype('uint8')).convert("RGB")) for img in X_light_train])
X_light_test_tensor = torch.stack([transform(Image.fromarray(img.astype('uint8')).convert("RGB")) for img in X_light_test])

In [121]:
train_dataset = TensorDataset(X_light_train_tensor, y_light_train_tensor)
test_dataset = TensorDataset(X_light_test_tensor, y_light_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)

In [122]:
model = models.efficientnet_b0(pretrained=True)
model.classifier[1] = nn.Linear(model.classifier[1].in_features, 2)  # 2 classes
model = model.to(device)

/opt/miniconda3/envs/adae-py310/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/miniconda3/envs/adae-py310/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [123]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

In [ ]:
epochs = 50
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    correct = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        preds = torch.argmax(outputs, 1)
        correct += (preds == labels).sum().item()

    accuracy = correct / len(train_dataset)
    print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss:.4f}, Accuracy: {accuracy:.4f}")

Epoch 1/10, Loss: 3.8021, Accuracy: 0.7204
Epoch 2/10, Loss: 3.0424, Accuracy: 0.8602
Epoch 3/10, Loss: 2.3581, Accuracy: 0.9355
Epoch 4/10, Loss: 1.7895, Accuracy: 0.9785
Epoch 5/10, Loss: 1.3360, Accuracy: 0.9624
Epoch 6/10, Loss: 0.9876, Accuracy: 0.9785
Epoch 7/10, Loss: 0.7328, Accuracy: 0.9839
Epoch 8/10, Loss: 0.5919, Accuracy: 0.9785
Epoch 9/10, Loss: 0.5132, Accuracy: 0.9839
Epoch 10/10, Loss: 0.4341, Accuracy: 0.9946


In [125]:
model.eval()
correct = 0

all_preds = []
all_labels = []

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        preds = torch.argmax(outputs, 1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())
        correct += (preds == labels).sum().item()

test_accuracy = correct / len(test_dataset)
print(f"Test Accuracy: {test_accuracy:.4f}")

print("Confusion Matrix:")
print(confusion_matrix(all_labels, all_preds))

print("\nClassification Report:")
print(classification_report(all_labels, all_preds, target_names=["Melanoma (0)", "Benign (1)"]))

Test Accuracy: 0.7234
Confusion Matrix:
[[ 2 13]
 [ 0 32]]

Classification Report:
              precision    recall  f1-score   support

Melanoma (0)       1.00      0.13      0.24        15
  Benign (1)       0.71      1.00      0.83        32

    accuracy                           0.72        47
   macro avg       0.86      0.57      0.53        47
weighted avg       0.80      0.72      0.64        47



In [47]:
torch.save(model.state_dict(), "efficientnet_melanoma_light.pth")

In [80]:
# medium model
folder = "ddi + pad"
file_names = three_dataset_medium["img_id"]
images = []
for file in file_names:
    path = os.path.join(folder, file)
    image = PIL.Image.open(path)
    images.append(np.array(image_resize(image)).flatten())
X_medium = np.stack(images)
#print(X_medium.shape)

y_medium = three_dataset_medium["label"]
le = LabelEncoder()
y_medium_encoded = le.fit_transform(y_medium)

X_medium_train, X_medium_test, y_medium_train, y_medium_test = train_test_split(X_medium, y_medium_encoded, test_size=0.2, stratify=y_medium_encoded, random_state=42)

X_medium_train_tensor = torch.stack([transform(Image.fromarray(img.astype('uint8')).convert("RGB"))for img in X_medium_train])
X_medium_test_tensor = torch.stack([transform(Image.fromarray(img.astype('uint8')).convert("RGB"))for img in X_medium_test])
y_medium_train_tensor = torch.tensor(y_medium_train, dtype=torch.long)
y_medium_test_tensor = torch.tensor(y_medium_test, dtype=torch.long)

train_dataset_medium = TensorDataset(X_medium_train_tensor, y_medium_train_tensor)
test_dataset_medium = TensorDataset(X_medium_test_tensor, y_medium_test_tensor)
train_loader_medium = DataLoader(train_dataset_medium, batch_size=32, shuffle=True)
test_loader_medium = DataLoader(test_dataset_medium, batch_size=32)

model_medium = models.efficientnet_b0(pretrained=True)
model_medium.classifier[1] = nn.Linear(model_medium.classifier[1].in_features, 2)
model_medium = model_medium.to(device)

class_weights = [233 / 33, 233 / 200]
weights_tensor = torch.tensor(class_weights, dtype=torch.float32).to(device)
criterion = nn.CrossEntropyLoss(weight=weights_tensor)
optimizer = optim.Adam(model_medium.parameters(), lr=1e-4)

epochs = 10
for epoch in range(epochs):
    model_medium.train()
    running_loss = 0.0
    correct = 0
    for images, labels in train_loader_medium:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model_medium(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        preds = torch.argmax(outputs, 1)
        correct += (preds == labels).sum().item()
    accuracy = correct / len(train_dataset_medium)
    print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss:.4f}, Accuracy: {accuracy:.4f}")

model_medium.eval()
correct = 0

all_preds = []
all_labels = []

with torch.no_grad():
    for images, labels in test_loader_medium:
        images, labels = images.to(device), labels.to(device)
        outputs = model_medium(images)
        preds = torch.argmax(outputs, 1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())
        correct += (preds == labels).sum().item()
test_accuracy = correct / len(test_dataset_medium)
print(f"Test Accuracy: {test_accuracy:.4f}")

print("Confusion Matrix:")
print(confusion_matrix(all_labels, all_preds))

print("\nClassification Report:")
print(classification_report(all_labels, all_preds, target_names=["Melanoma (0)", "Benign (1)"]))


/opt/miniconda3/envs/adae-py310/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/miniconda3/envs/adae-py310/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1/10, Loss: 4.2103, Accuracy: 0.5753
Epoch 2/10, Loss: 3.9344, Accuracy: 0.6398
Epoch 3/10, Loss: 3.8387, Accuracy: 0.7473
Epoch 4/10, Loss: 3.6421, Accuracy: 0.7849
Epoch 5/10, Loss: 3.3445, Accuracy: 0.8118
Epoch 6/10, Loss: 2.8894, Accuracy: 0.8817
Epoch 7/10, Loss: 2.6972, Accuracy: 0.8817
Epoch 8/10, Loss: 2.4286, Accuracy: 0.8871
Epoch 9/10, Loss: 1.8792, Accuracy: 0.9355
Epoch 10/10, Loss: 1.9636, Accuracy: 0.8978
Test Accuracy: 0.8298
Confusion Matrix:
[[ 2  5]
 [ 3 37]]

Classification Report:
              precision    recall  f1-score   support

Melanoma (0)       0.40      0.29      0.33         7
  Benign (1)       0.88      0.93      0.90        40

    accuracy                           0.83        47
   macro avg       0.64      0.61      0.62        47
weighted avg       0.81      0.83      0.82        47



In [77]:
all_preds = []
all_labels = []

model_medium.eval()
with torch.no_grad():
    for images, labels in test_loader_medium:
        images, labels = images.to(device), labels.to(device)
        outputs = model_medium(images)
        preds = torch.argmax(outputs, dim=1)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

print("Confusion Matrix:")
print(confusion_matrix(all_labels, all_preds))

print("\nClassification Report:")
print(classification_report(all_labels, all_preds, target_names=["Melanoma (0)", "Benign (1)"]))


Confusion Matrix:
[[ 1  6]
 [ 1 39]]

Classification Report:
              precision    recall  f1-score   support

Melanoma (0)       0.50      0.14      0.22         7
  Benign (1)       0.87      0.97      0.92        40

    accuracy                           0.85        47
   macro avg       0.68      0.56      0.57        47
weighted avg       0.81      0.85      0.81        47



In [68]:
torch.save(model_medium.state_dict(), "efficientnet_melanoma_medium.pth")

In [69]:
# dark model
folder = "ddi + pad"
file_names = three_dataset_dark["img_id"]
images = []

for file in file_names:
    path = os.path.join(folder, file)
    image = PIL.Image.open(path)
    images.append(np.array(image_resize(image)).flatten())
X_dark = np.stack(images)
y_dark = three_dataset_dark["label"]

le = LabelEncoder()
y_dark_encoded = le.fit_transform(y_dark)

X_dark_train, X_dark_test, y_dark_train, y_dark_test = train_test_split(X_dark, y_dark_encoded, test_size=0.2, stratify=y_dark_encoded, random_state=42)

X_dark_train_tensor = torch.stack([transform(Image.fromarray(img.astype('uint8')).convert("RGB")) for img in X_dark_train])
X_dark_test_tensor = torch.stack([transform(Image.fromarray(img.astype('uint8')).convert("RGB")) for img in X_dark_test])

y_dark_train_tensor = torch.tensor(y_dark_train, dtype=torch.long)
y_dark_test_tensor = torch.tensor(y_dark_test, dtype=torch.long)

train_dataset_dark = TensorDataset(X_dark_train_tensor, y_dark_train_tensor)
test_dataset_dark = TensorDataset(X_dark_test_tensor, y_dark_test_tensor)

train_loader_dark = DataLoader(train_dataset_dark, batch_size=32, shuffle=True)
test_loader_dark = DataLoader(test_dataset_dark, batch_size=32)

model_dark = models.efficientnet_b0(pretrained=True)
model_dark.classifier[1] = nn.Linear(model_dark.classifier[1].in_features, 2)
model_dark = model_dark.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_dark.parameters(), lr=1e-4)

epochs = 10
for epoch in range(epochs):
    model_dark.train()
    running_loss = 0.0
    correct = 0
    for images, labels in train_loader_dark:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model_dark(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        preds = torch.argmax(outputs, 1)
        correct += (preds == labels).sum().item()
    accuracy = correct / len(train_dataset_dark)
    print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss:.4f}, Accuracy: {accuracy:.4f}")

model_dark.eval()
correct = 0
with torch.no_grad():
    for images, labels in test_loader_dark:
        images, labels = images.to(device), labels.to(device)
        outputs = model_dark(images)
        preds = torch.argmax(outputs, 1)
        correct += (preds == labels).sum().item()
test_accuracy = correct / len(test_dataset_dark)
print(f"Test Accuracy: {test_accuracy:.4f}")

/opt/miniconda3/envs/adae-py310/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/miniconda3/envs/adae-py310/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1/10, Loss: 3.9566, Accuracy: 0.6022
Epoch 2/10, Loss: 3.3506, Accuracy: 0.8226
Epoch 3/10, Loss: 2.7409, Accuracy: 0.8925
Epoch 4/10, Loss: 2.2982, Accuracy: 0.9140
Epoch 5/10, Loss: 1.8200, Accuracy: 0.9409
Epoch 6/10, Loss: 1.5030, Accuracy: 0.9677
Epoch 7/10, Loss: 1.2972, Accuracy: 0.9624
Epoch 8/10, Loss: 1.0870, Accuracy: 0.9677
Epoch 9/10, Loss: 0.9223, Accuracy: 0.9677
Epoch 10/10, Loss: 0.6797, Accuracy: 0.9839
Test Accuracy: 0.8511


In [71]:
print("Medium test set size:", len(test_dataset_medium))
print("Dark test set size:", len(test_dataset_dark))

Medium test set size: 47
Dark test set size: 47


In [75]:
melanoma_count = np.sum(y_medium_test == 0)
not_melanoma_count = np.sum(y_medium_test == 1)

print(f"Total test samples: {len(y_medium_test)}")
print(f"Melanoma count (label=0): {melanoma_count}")
print(f"Benign count (label=1): {not_melanoma_count}")

Total test samples: 47
Melanoma count (label=0): 7
Benign count (label=1): 40


In [60]:
torch.save(model_dark.state_dict(), "efficientnet_melanoma_dark.pth")

In [70]:
# total model
folder = "ddi + pad"
file_names = three_dataset["img_id"]
images = []

for file in file_names:
    path = os.path.join(folder, file)
    image = PIL.Image.open(path)
    images.append(np.array(image_resize(image)).flatten())
X_total = np.stack(images)
y_total = three_dataset["label"]

le = LabelEncoder()
y_total_encoded = le.fit_transform(y_total)

X_total_train, X_total_test, y_total_train, y_total_test = train_test_split(X_total, y_total_encoded, test_size=0.2, stratify=y_total_encoded, random_state=42)

X_total_train_tensor = torch.stack([transform(Image.fromarray(img.astype('uint8')).convert("RGB")) for img in X_total_train])
X_total_test_tensor = torch.stack([transform(Image.fromarray(img.astype('uint8')).convert("RGB")) for img in X_total_test])

y_total_train_tensor = torch.tensor(y_total_train, dtype=torch.long)
y_total_test_tensor = torch.tensor(y_total_test, dtype=torch.long)

train_dataset_total = TensorDataset(X_total_train_tensor, y_total_train_tensor)
test_dataset_total = TensorDataset(X_total_test_tensor, y_total_test_tensor)

train_loader_total = DataLoader(train_dataset_total, batch_size=32, shuffle=True)
test_loader_total = DataLoader(test_dataset_total, batch_size=32)

model_total = models.efficientnet_b0(pretrained=True)
model_total.classifier[1] = nn.Linear(model_total.classifier[1].in_features, 2)
model_total = model_total.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_total.parameters(), lr=1e-4)

epochs = 10
for epoch in range(epochs):
    model_total.train()
    running_loss = 0.0
    correct = 0
    for images, labels in train_loader_total:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model_total(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        preds = torch.argmax(outputs, 1)
        correct += (preds == labels).sum().item()
    accuracy = correct / len(train_dataset_total)
    print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss:.4f}, Accuracy: {accuracy:.4f}")

model_total.eval()
correct = 0
with torch.no_grad():
    for images, labels in test_loader_total:
        images, labels = images.to(device), labels.to(device)
        outputs = model_total(images)
        preds = torch.argmax(outputs, 1)
        correct += (preds == labels).sum().item()
test_accuracy = correct / len(test_dataset_total)
print(f"Test Accuracy: {test_accuracy:.4f}")

/opt/miniconda3/envs/adae-py310/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/miniconda3/envs/adae-py310/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1/10, Loss: 10.4098, Accuracy: 0.7621
Epoch 2/10, Loss: 7.1601, Accuracy: 0.8623
Epoch 3/10, Loss: 5.7421, Accuracy: 0.8748
Epoch 4/10, Loss: 5.9962, Accuracy: 0.8694
Epoch 5/10, Loss: 4.6274, Accuracy: 0.8980
Epoch 6/10, Loss: 3.8475, Accuracy: 0.9141
Epoch 7/10, Loss: 2.7587, Accuracy: 0.9517
Epoch 8/10, Loss: 2.4011, Accuracy: 0.9606
Epoch 9/10, Loss: 1.7025, Accuracy: 0.9857
Epoch 10/10, Loss: 1.4212, Accuracy: 0.9911
Test Accuracy: 0.8857


In [62]:
torch.save(model_total.state_dict(), "efficientnet_melanoma_total.pth")

In [49]:
model_light = models.efficientnet_b0(pretrained=False)
model_light.classifier[1] = nn.Linear(model_light.classifier[1].in_features, 2)
model_light.load_state_dict(torch.load("efficientnet_melanoma_light.pth"))
model_light = model.to(device)
model_light.eval()

/var/folders/qz/3gzbdppx7f3fd_hs65_qss5c0000gn/T/ipykernel_9662/318641267.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_light.load_state_dict(torch.load("efficie

EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): Conv2dNormActivat

In [ ]:
# medium set
folder = "ddi + pad"
file_names = three_dataset_medium["img_id"]
images = []
for file in file_names:
    path = os.path.join(folder, file)
    image = PIL.Image.open(path)
    #print(image.size) # Returns (width, height)
    images.append(np.array(image_resize(image)).flatten())
X_medium = np.stack(images)
#print(X_medium.shape)

y_medium = three_dataset_medium["label"]
le = LabelEncoder()
y_medium_encoded = le.fit_transform(y_medium)

X_medium_tensor = torch.stack([transform(Image.fromarray(img.astype("uint8")).convert("RGB"))for img in X_medium])
y_medium_tensor = torch.tensor(y_medium_encoded, dtype=torch.long)

test_dataset_medium = TensorDataset(X_medium_tensor, y_medium_tensor)
test_loader_medium = DataLoader(test_dataset_medium, batch_size=32)

In [ ]:
# test medium set in light model
correct = 0
with torch.no_grad():
    for images, labels in test_loader_medium:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        preds = torch.argmax(outputs, 1)
        correct += (preds == labels).sum().item()

accuracy_medium = correct / len(test_dataset_medium)
print(f"Test Accuracy on medium skin tone: {accuracy_medium:.4f}")

Test Accuracy on medium skin tone: 0.7039


In [ ]:
# dark set
folder = "ddi + pad"
file_names = three_dataset_dark["img_id"]
images = []
for file in file_names:
    path = os.path.join(folder, file)
    image = PIL.Image.open(path)
    #print(image.size) # Returns (width, height)
    images.append(np.array(image_resize(image)).flatten())
X_dark = np.stack(images)
#print(X_dark.shape)

y_dark = three_dataset_dark["label"]
le = LabelEncoder()
y_dark_encoded = le.fit_transform(y_dark)

X_dark_tensor = torch.stack([
    transform(Image.fromarray(img.astype("uint8")).convert("RGB")) 
    for img in X_dark
])
y_dark_tensor = torch.tensor(y_dark_encoded, dtype=torch.long)

test_dataset_dark = TensorDataset(X_dark_tensor, y_dark_tensor)
test_loader_dark = DataLoader(test_dataset_dark, batch_size=32)

In [ ]:
# test dark set in light model
correct = 0
with torch.no_grad():
    for images, labels in test_loader_dark:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        preds = torch.argmax(outputs, 1)
        correct += (preds == labels).sum().item()

accuracy_dark = correct / len(test_dataset_dark)
print(f"Test Accuracy on dark skin tone: {accuracy_dark:.4f}")

Test Accuracy on dark skin tone: 0.6910


In [ ]:
# light set
folder = "ddi + pad"
file_names = three_dataset_light["img_id"]
images = []
for file in file_names:
    path = os.path.join(folder, file)
    image = PIL.Image.open(path)
    images.append(np.array(image_resize(image)).flatten())
X_light = np.stack(images)

y_light = three_dataset_light["label"]
le = LabelEncoder()
y_light_encoded = le.fit_transform(y_light)

X_light_tensor = torch.stack([transform(Image.fromarray(img.astype("uint8")).convert("RGB"))for img in X_medium])
y_light_tensor = torch.tensor(y_light_encoded, dtype=torch.long)

test_dataset_light = TensorDataset(X_light_tensor, y_light_tensor)


In [64]:
# light + medium + dark set
folder = "ddi + pad"
file_names = three_dataset["img_id"]
images = []
for file in file_names:
    path = os.path.join(folder, file)
    image = PIL.Image.open(path)
    images.append(np.array(image_resize(image)))
X_total = np.stack(images)

y_total = three_dataset["label"]
le = LabelEncoder()
y_total_encoded = le.fit_transform(y_total)

X_total_tensor = torch.stack([transform(Image.fromarray(img.astype("uint8")).convert("RGB"))for img in X_total])
y_total_tensor = torch.tensor(y_total_encoded, dtype=torch.long)

test_dataset_total = TensorDataset(X_total_tensor, y_total_tensor)